In [1]:
import re
import os
import pandas as pd
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt


def create_bins(lower_bound, width, quantity):
    """ create_bins returns an equal-width (distance) partitioning. 
        It returns an ascending list of tuples, representing the intervals.
        A tuple bins[i], i.e. (bins[i][0], bins[i][1])  with i > 0 
        and i < quantity, satisfies the following conditions:
            (1) bins[i][0] + width == bins[i][1]
            (2) bins[i-1][0] + width == bins[i][0] and
                bins[i-1][1] + width == bins[i][1]
    """
    
    bins = []
    for low in range(lower_bound, 
                     lower_bound + quantity*width + 1, width):
        bins.append((low, low+width))
    return bins

In [3]:
pdf = pd.read_csv("/home/brian/results-analysis/data/processed_data/2019-03-26_15:09-david-cross-roads-cv2x/vector.csv",usecols=["node","name", "vectime", "vecvalue"])

#print(pdf)

pdrs = []

nodes = pdf.node.unique()

pdf_dict = {}
pdf_dict["time"] = []
pdf_dict["rec"] = []
pdf_dict["distance"] = []
pdf_dict["failbutrec"] = []
pdf_dict["failduesci"] = []

overall_df = pd.DataFrame(pdf_dict)

In [10]:
pdf.node = pdf.node.str.replace("car", "")
pdf.name.unique()

array(['queueLength:vector', 'txState:vector', 'servingCell:vector',
       'sentPacketToLowerLayer:vector(packetBytes)',
       'alertSentMsg:vector', 'sentPk:vector(packetBytes)',
       'receivedPacketFromUpperLayer:vector(packetBytes)',
       'sentPacketToUpperLayer:vector(packetBytes)',
       'numAirFramesWithSCIsReceivedSignal:vector',
       'distanceSignal:vector',
       'numAirFramesWithTBsReceivedSignal:vector',
       'numSCIAndTBPairsSignal:vector',
       'receivedPacketFromLowerLayer:vector(packetBytes)',
       'macDelayDl:vector',
       'numAirFramesWithSCIsNotReceivedSignal:vector',
       'numAirFramesWithTBsNotReceivedSignal:vector',
       'rcvdPk:vector(packetBytes)', 'droppedPkWrongPort:vector(count)'],
      dtype=object)

In [3]:
for node in nodes:

    # print("Processing Node {} of {}".format(node+1, len(nodes)))

    pdf2 = pdf[pdf["node"] == node]

    recieved_times = pdf2[pdf2["name"] == "tbsReceived:vector"]["vectime"].values
    recieved_val = pdf2[pdf2["name"] == "tbsReceived:vector"]["vecvalue"].values

    dist = pdf2[pdf2["name"] == "txRxDistance:vector"]["vecvalue"].values
    dist_time = pdf2[pdf2["name"] == "txRxDistance:vector"]["vectime"].values

    failbutrec = pdf2[pdf2["name"] == "tbFailedButSCIReceived:vector"]["vecvalue"].values
    failbutrec_time = pdf2[pdf2["name"] == "tbFailedButSCIReceived:vector"]["vectime"].values

    failduesci = pdf2[pdf2["name"] == "tbsFailedDueToNoSCI:vector"]["vecvalue"].values
    failduesci_time = pdf2[pdf2["name"] == "tbsFailedDueToNoSCI:vector"]["vectime"].values

    #print(pdf2)

    #final_res = (recieved - failbutrec - failduesci)/ recieved *100

    recieved_times = np.array(recieved_times[0].split(" ")).astype("float")
    recieved_val = np.array(recieved_val[0].split(" ")).astype("float")

    dist = np.array(dist[0].split(" ")).astype("float")
    dist_time = np.array(dist_time[0].split(" ")).astype("float")

    failbutrec = np.array(failbutrec[0].split(" ")).astype("float")
    failbutrec_time = np.array(failbutrec_time[0].split(" ")).astype("float")

    failduesci = np.array(failduesci[0].split(" ")).astype("float")
    failduesci_time = np.array(failduesci_time[0].split(" ")).astype("float")

    recieved = []
    failbutrec_list = []
    failduesci_list = []
    dist_list = []

    # print(failbutrec)
    # print(failbutrec_time)

    for inst in recieved_times:

        itemindex_failbutrec = np.where(failbutrec_time == inst)
        if itemindex_failbutrec[0].size != 0:
            failbutrec_list.append(failbutrec[itemindex_failbutrec[0][0]])
        else:
            failbutrec_list.append(0)

        itemindex_failduesci = np.where(failduesci_time == inst)
        if itemindex_failduesci[0].size != 0:
            failduesci_list.append(failduesci[itemindex_failduesci[0][0]])
        else:
            failduesci_list.append(0)

        itemindex_disttime = np.where(dist_time == inst)
        if itemindex_disttime[0].size != 0:
            dist_list.append(dist[itemindex_disttime[0][0]])
        else:
            dist_list.append(-1)

        itemindex_rectime = np.where(recieved_times == inst)
        recieved.append(recieved_val[itemindex_rectime[0][0]])



    pdf_dict = {}
    pdf_dict["time"] = recieved_times
    pdf_dict["rec"] = recieved
    pdf_dict["distance"] = dist_list
    pdf_dict["failbutrec"] = failbutrec_list
    pdf_dict["failduesci"] = failduesci_list
    pdf3 = pd.DataFrame(pdf_dict)

    pdf_500m = pdf3[pdf3["distance"] <= 500]
    if overall_df.size == 0:
        overall_df = pdf_500m
    else:
        overall_df.append(pdf_500m)

In [4]:
print(overall_df.describe())

               time      rec      distance    failbutrec    failduesci
count  10938.000000  10938.0  10938.000000  10938.000000  10938.000000
mean      34.187766      1.0    174.741061      0.062443      0.046626
std       11.110904      0.0    122.158255      0.241969      0.210847
min        3.171000      1.0      5.137039      0.000000      0.000000
25%       26.868000      1.0     77.376637      0.000000      0.000000
50%       36.102500      1.0    137.307088      0.000000      0.000000
75%       43.314000      1.0    247.863687      0.000000      0.000000
max       49.994000      1.0    499.164881      1.000000      1.000000


In [15]:
overall_df.distance.unique()

array([  8.10032228,   8.25932358,   5.13703878,   6.07639117,
        31.72518129,  29.13567462,  32.54318132,  28.12764114,
        32.82315078,   7.22310422,  34.16648104,  35.64940805,
         8.59630225,  29.06581007,  35.99554876,  38.84522119,
        29.58887909,  10.30883579,  70.12847289,  73.32613528,
        37.44065713,  30.39184763,  12.51025529,  74.1053842 ,
        72.0345361 ,  41.43009364,  39.03977314,  71.48527646,
        43.16833883,  13.79715629,  74.43695227, 100.76291933,
       100.05270498,  98.06109334,  29.71089599,  40.31632295,
        71.77673701,  46.22372031,  30.32202929, 102.79974599,
        16.21934973,  75.44704036, 101.06097821,  97.42493177,
        42.6798337 ,  48.64063889,  30.07939248, 105.27238094,
        17.64500135,  76.58363452, 102.37223533,  96.23228365,
        70.8641849 ,  72.37170817,  44.37728354,  72.5316946 ,
        51.35651091,  31.13418921,  19.69574322,  72.83604938,
       103.20328645,  96.33755734, 145.25919361,  77.94

In [5]:
bins = create_bins(lower_bound=0, width=10, quantity=49)

In [6]:
for interval in bins:
    lower_b = interval[0]
    upper_b = interval[1]
    pdf_temp = pdf_500m[(pdf_500m["distance"] >= lower_b) & (pdf_500m["distance"] < upper_b)]
    pdr = ((pdf_temp["rec"].sum() - pdf_temp["failbutrec"].sum() - pdf_temp["failduesci"].sum()) / pdf_temp["rec"].sum()) *100
    print ("{}-{} : {}".format(lower_b, upper_b, pdr))

0-10 : 100.0
10-20 : nan
20-30 : 100.0
30-40 : nan
40-50 : nan
50-60 : 100.0
60-70 : 100.0
70-80 : 100.0
80-90 : 100.0
90-100 : nan
100-110 : nan
110-120 : nan
120-130 : nan
130-140 : 100.0
140-150 : nan
150-160 : 100.0
160-170 : 100.0
170-180 : nan
180-190 : nan
190-200 : 100.0
200-210 : 100.0
210-220 : 100.0
220-230 : nan
230-240 : 100.0
240-250 : nan
250-260 : 100.0
260-270 : nan
270-280 : nan
280-290 : 100.0
290-300 : nan
300-310 : nan
310-320 : 95.0
320-330 : nan
330-340 : 98.33333333333333
340-350 : 100.0
350-360 : 30.0
360-370 : 36.666666666666664
370-380 : 100.0
380-390 : nan
390-400 : nan
400-410 : 2.941176470588235
410-420 : 0.0
420-430 : 3.7735849056603774
430-440 : 95.0
440-450 : 33.33333333333333
450-460 : 0.0
460-470 : 50.526315789473685
470-480 : nan
480-490 : 50.0
490-500 : nan


/home/brian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
